In [4]:
from scipy.io import arff


In [9]:
file_path ='datasets/mnist_784.arff'

In [24]:
data,meta = arff.loadarff(file_path)

In [72]:
import numpy as np
mnist = np.array(data)
mnist[0]

(0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 18., 18., 18., 126., 136., 175., 26., 166., 255., 247., 127., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 30., 36., 94., 154., 170., 253., 253., 253., 253., 253., 225., 172., 253., 242., 195., 64., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 49., 238., 253., 253., 253., 253., 253., 253., 253., 253., 251., 93., 82., 82., 56., 39., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 18.

In [73]:
import pandas as pd
mnist_df = pd.DataFrame(mnist)
mnist_df.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'5'
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'0'
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'4'
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'1'
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'9'


In [49]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

some_digit = X[1, :]
some_digit_image = some_digit.reshape(28,28)

InvalidIndexError: (1, slice(None, None, None))